### Using manual feature-map

### Naive binarization (works)

In [259]:
%reset -f

from collections import defaultdict
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

def remapOnFeatureMap2(data, featureMap2):
    remapOnFeatureMap2 = []
    
    for i, row in enumerate(data):
        newRow = []
        for j, d in enumerate(row):
            if j not in [0,7]:
                if (j,d) in featureMap2:
                    newRow.append(featureMap2[(j,d)])
        remapOnFeatureMap2.append(newRow)
    
    binRemapped2 = np.zeros((len(data), len(featureMap2)), dtype=float)
    for i, row in enumerate(remapOnFeatureMap2):
        for j, d in enumerate(row):
            if j not in [0,7]:
                binRemapped2[i][d] = 1
            else:
                binRemapped2[i][j] = d
        binRemapped2[i][-1] = 1
    return binRemapped2

def removeNA(data):
    for i, row in enumerate(data):
        for j, d in enumerate(row):
            if d=='NA':
                data[i][j] = 0
    return data

def removeNAFromDF(data):
    for col in data.columns:
        if col not in binTrainDF.columns:
            data = data.drop(col, axis = 1)
    print('After dropping extra cols from data ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in binTrainDF.columns:
        if col not in data.columns:
            data[col] = 0
    print('After setting cols in data which are not in train to zero ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in data.columns:
        data[col] = data[col].fillna(data[col].mean())
#         data[col] = data[col].fillna(0)
    
    return data

def writePredictions(ids, preds):
    final = [str(i)+','+str(p) for i,p in zip(ids, preds)]
    with open('submission.csv', 'w') as output:
        output.write('Id,SalePrice\n')
        for x in final:
            output.write(str(x))
            output.write('\n')

if __name__ == '__main__':
    trainFile, devFile, testFile = 'my_train.csv', 'my_dev.csv', 'test.csv'
    
    train = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][1:])
    dev = np.array([s.strip().split(',')[1:-1] for s in open(devFile).readlines()][1:])
    test = np.array([s.strip().split(',')[1:] for s in open(testFile).readlines()][1:])
#     print('len of dev {} and len of test {}'.format(len(dev), len(test)))

    header = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][0])
#     print([(h,val) for h, val in enumerate(header)])
    
    trainIDs = [s.strip().split(',')[0] for s in open(trainFile).readlines()][1:]
    devIDs = [s.strip().split(',')[0] for s in open(devFile).readlines()][1:]
    testIDs = [s.strip().split(',')[0] for s in open(testFile).readlines()][1:]
    
#     trainY = map(float, [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = np.array(list(trainY))
#     devY = map(float, [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = np.array(list(devY))

    train = removeNA(train)
    dev = removeNA(dev)
    
    featureMap = defaultdict()
    featureRemap = defaultdict()
    featureCategories = defaultdict(list)
    for i, row in enumerate(train):
        for j, d in enumerate(row):
            feature = (j,d)
            if feature not in featureMap:
                featureMap[feature] = len(featureMap)
                featureRemap[len(featureMap)-1] = (feature)
                featureCategories[j].append(d)
    
    print('Len of feature map: {}'.format(len(featureMap)))
    
    binTrain = remapOnFeatureMap2(train, featureMap)
    binDev = remapOnFeatureMap2(dev, featureMap)
    binTest = remapOnFeatureMap2(test, featureMap)
    
    model = LinearRegression().fit(X=binTrain, y=trainY)
    coefs = model.coef_
    intercept = model.intercept_
    preds = model.predict(binDev)
    
    print('len(coefs): {} coefs: {}'.format(len(coefs), coefs))
    sortedCoefs = np.argsort(coefs)
    print('Sorted most positive coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][:10]))
    print('Sorted most negative coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][-10:]))
    print('mse: {}'.format(np.sqrt(mean_squared_error(devY, preds))))
    print('intercept: {}'.format(intercept))
    
    preds = [np.exp(s) for s in preds]
    print('predictions after exponentiation: {}'.format(list(zip(preds,np.exp(devY)))))

Len of feature map: 7226
len(coefs): 7226 coefs: [-8.30274722e-06  2.65356156e-03 -3.55315489e-02 ...  9.06097491e-03
  9.06097491e-03  0.00000000e+00]
Sorted most positive coefs: [(45, '968'), (16, '3'), (67, '236'), (11, 'IDOTRR'), (35, '311'), (3, '8281'), (17, '3'), (33, '50'), (16, '2'), (36, '430')]
Sorted most negative coefs: [(11, 'NoRidge'), (36, '720'), (21, 'WdShngl'), (11, 'Crawfor'), (45, '1192'), (16, '8'), (43, '472'), (11, 'StoneBr'), (48, '3'), (16, '9')]
mse: 0.15764496505166586
intercept: 12.228006625696883
predictions after exponentiation: [(133819.2871723195, 117999.99999999991), (358916.8596011563, 325300.0000000001), (129400.48627464366, 149349.99999999988), (137822.42759276184, 152999.99999999994), (140602.66894037768, 170000.0), (98925.89982103335, 109999.99999999991), (172572.68153355754, 180499.99999999988), (103467.38504568982, 136500.0000000001), (210479.85526023706, 173999.99999999997), (103695.11939825317, 128949.99999999988), (137332.75862619883, 136899.

### Smart Binarization

In [275]:
%reset -f

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

def writePredictions(ids, preds):
    final = [str(i)+','+str(p) for i,p in zip(ids, preds)]
    with open('submission.csv', 'w') as output:
        output.write('Id,SalePrice\n')
        for x in final:
            output.write(str(x))
            output.write('\n')

if __name__ == '__main__':
    trainDF, devDF, testDF = pd.read_csv('train.csv'), pd.read_csv('my_dev.csv'), pd.read_csv('test.csv')
    trainYDF, devYDF = list(map(lambda s : np.log(s), trainDF['SalePrice'])), list(map(lambda s : np.log(s), devDF['SalePrice']))
    trainIDs, devIDs, testIDs = trainDF['Id'], devDF['Id'], testDF['Id']
    
#     num = [0,2,3,16,17,18,19,33,35,36,37,42,43,44,45,46,47,48,49,50,51,53,55,60,61,65,66,67,69,70,74,75,76]
#     cat = [1,4,5,6,7,8,9,10,11,12,13,14,15,20,21,22,23,24,25,26,27,28,29,30,31,32,34,38,39,40,41,52,54,56,57,58,59,62,63,64,68,71,72,73,77,78]
    
    trainLen = trainDF.shape[0]
    print('train shape: {}'.format(trainDF.shape))
    print('dev shape: {}'.format(devDF.shape))
    print('test shape: {}'.format(testDF.shape))
    
    trainDF.drop(['Id', 'SalePrice'], inplace=True, axis = 1)
    devDF.drop(['Id', 'SalePrice'], inplace=True, axis = 1)
    testDF.drop('Id', inplace=True, axis = 1)
    
    mergedDF = pd.concat([trainDF, devDF], axis = 0).reset_index(drop=True)
#     mergedDF = pd.concat([trainDF, testDF], axis = 0).reset_index(drop=True)
    
    print('merged shape after dropping Id and SalePrice: {}'.format(mergedDF.shape))
    
    print('------------------------------------------------------------------------')
        
    num = trainDF.select_dtypes(include=np.number).columns
    cat = trainDF.select_dtypes(exclude=np.number).columns
    print('num of numerical: {} num of cat: {} '.format(len(num), len(cat)))
        
    binMergedDF = pd.get_dummies(mergedDF, columns=cat).fillna(0)
    print('binMergedDF shape after get_dummies: {}'.format(binMergedDF.shape))
    
    binTrainDF = binMergedDF.iloc[:trainLen,:]
    binDevDF = binMergedDF.iloc[trainLen:,:]
#     binTestDF = binMergedDF.iloc[trainLen:,:]
    
    print('binTrainDF shape: {} binDevDF shape: {}'.format(binTrainDF.shape, binDevDF.shape))
#     print('binTrainDF shape: {} binTestDF shape: {}'.format(binTrainDF.shape, binTestDF.shape))

    model = LinearRegression().fit(X=binTrainDF, y=trainYDF)
    preds = model.predict(binDevDF)
#     preds = model.predict(binTestDF)
    coefs = model.coef_
#     print('RMSLE: {}'.format(np.sqrt(mean_squared_error(devYDF, preds))))
    print('Weight for bias feature: {}'.format(model.intercept_))
    print('coefs: {}'.format())
    sortedCoefs = np.argsort(coefs)
    print('Sorted most positive coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][:10]))
    print('Sorted most negative coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][-10:]))

    temp = [(i,x) for (i,x) in enumerate(list(zip(np.exp(devYDF),np.exp(preds))))]
    print('zipping vals: {}'.format(temp))

#     print('testPreds: {}'.format(list(zip(testIDs,preds))))
#     writePredictions(testIDs,np.exp(preds))

train shape: (1460, 81)
dev shape: (146, 81)
test shape: (1459, 80)
merged shape after dropping Id and SalePrice: (2919, 79)
------------------------------------------------------------------------
num of numerical: 36 num of cat: 43 
binMergedDF shape after get_dummies: (2919, 288)
binTrainDF shape: (1460, 288) binTestDF shape: (1459, 288)
Weight for bias feature: 9.300898790239067
testPreds: [(1461, 11.706699536806564), (1462, 12.026247782001615), (1463, 12.117954058524022), (1464, 12.221337671136867), (1465, 12.179559757865452), (1466, 12.055547800387458), (1467, 12.09376541716649), (1468, 12.00465340781026), (1469, 12.18428048394776), (1470, 11.701325371612695), (1471, 12.02983686742115), (1472, 11.505514407798838), (1473, 11.465171919034606), (1474, 11.890603535358379), (1475, 11.657417662448662), (1476, 12.800687882616408), (1477, 12.443531575879444), (1478, 12.619603215218437), (1479, 12.610041225240694), (1480, 13.21362318030456), (1481, 12.712103689356478), (1482, 12.236711792

### Trying manual feature -map to get naive and smart binarization

In [179]:
%reset -f

from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import traceback

def getFeatureMap(data):
    naiveFM, smartFM, naiveFeatureRemap, smartFeatureRemap = defaultdict(int), defaultdict(int), defaultdict(), defaultdict()
    
    for i,row in enumerate(data):
        for j,d in enumerate(row):
            feature = (j,d)
            if feature not in naiveFM:
                naiveFM[feature] = len(naiveFM)
                naiveFeatureRemap[len(naiveFM)-1] = feature
            if feature not in smartFM:
                if j not in numFeatures:
                    smartFM[feature] = len(smartFM)
                    smartFeatureRemap[len(smartFM)-1] = feature
    return naiveFM, smartFM, naiveFeatureRemap, smartFeatureRemap

def remapOnFeatureMap2(data, fm):
    remapOnFM = []
    
    for i, row in enumerate(data):
        newRow = []
        for j, d in enumerate(row):
            if j not in numFeatures:
                if (j,d) in fm:
                    newRow.append(fm[(j,d)])
            else:
                newRow.append(int(d))
        remapOnFM.append(newRow)
    
    try:
        binRemapped2 = np.zeros((len(data), len(fm)), dtype=float)
        for i, row in enumerate(remapOnFM):
            for j, d in enumerate(row):
                if j not in numFeatures:
                    binRemapped2[i][d] = 1
                else:
                    binRemapped2[i][j] = d
#             binRemapped2[i][-1] = 1
    except Exception as e: 
        print(e)
        traceback.print_exc()
    return binRemapped2

def removeNA(data):
    for i, row in enumerate(data):
        for j, d in enumerate(row):
            if d=='NA':
                data[i][j] = 0
    return data
                

if __name__ == '__main__':
    trainFile, devFile, testFile = 'my_train.csv', 'my_dev.csv', 'test.csv'
    
    train = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][1:])
    dev = np.array([s.strip().split(',')[1:-1] for s in open(devFile).readlines()][1:])
    test = np.array([s.strip().split(',')[1:] for s in open(testFile).readlines()][1:])
#     print('len of dev {} and len of test {}'.format(len(dev), len(test)))

    header = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][0])
#     print([(h,val) for h, val in enumerate(header)])
    
    trainIDs = [s.strip().split(',')[0] for s in open(trainFile).readlines()][1:]
    devIDs = [s.strip().split(',')[0] for s in open(devFile).readlines()][1:]
    testIDs = [s.strip().split(',')[0] for s in open(testFile).readlines()][1:]
    
#     trainY = map(float, [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = np.array(list(trainY))
#     devY = map(float, [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = np.array(list(devY))
    
    numFeatures = [0,2,3,16,17,18,19,33,35,36,37,42,43,44,45,46,47,48,49,50,51,53,55,60,61,65,66,67,69,70,74,75,76]
    catFeatures = [1,4,5,6,7,8,9,10,11,12,13,14,15,20,21,22,23,24,25,26,27,28,29,30,31,32,34,38,39,40,41,52,54,56,57,58,59,62,63,64,68,71,72,73,77,78]
    
    train = removeNA(train)
    dev = removeNA(dev)
#     test = removeNA(test)
    
    naiveFeatureMap, smartFeatureMap, naiveFeatureRemap, smartFeatureRemap = getFeatureMap(train)
    print('naiveFeatureMap: {} naiveFeatureRemap: {} smartFeatureMap: {} smartFeatureRemap: {}'.format(len(naiveFeatureMap), len(naiveFeatureRemap), len(smartFeatureMap), len(smartFeatureRemap)))
    binTrain = remapOnFeatureMap2(train, naiveFeatureMap)
    binDev = remapOnFeatureMap2(dev, naiveFeatureMap)
#     binTest = remapOnFeatureMap2(test, naiveFeatureMap)
    
    model = LinearRegression().fit(X=binTrain, y=trainY)
    coefs = model.coef_
    intercept = model.intercept_
    preds = model.predict(binDev)
    
    print('len(coefs): {} coefs: {}'.format(len(coefs), coefs))
    sortedCoefs = np.argsort(coefs)
    print('Sorted most positive coefs: {}'.format([naiveFeatureRemap[coef] for coef in sortedCoefs][:10]))
    print('Sorted most negative coefs: {}'.format([naiveFeatureRemap[coef] for coef in sortedCoefs][-10:]))
    print('mse: {}'.format(np.sqrt(mean_squared_error(devY, preds))))
    print('intercept: {}'.format(intercept))
    
    preds = [np.exp(s) for s in preds]
    print('predictions after exponentiation: {}'.format(list(zip(preds,np.exp(devY)))))

naiveFeatureMap: 7226 naiveFeatureRemap: 7226 smartFeatureMap: 685 smartFeatureRemap: 685
len(coefs): 7226 coefs: [2.35258627e-04 1.06505750e-01 5.24647821e-05 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
Sorted most positive coefs: [(21, 'ClyTile'), (25, '796'), (25, '762'), (13, 'RRAe'), (13, 'PosN'), (1, 'C (all)'), (63, 'Ex'), (71, 'Fa'), (25, '287'), (54, 'Maj2')]
Sorted most negative coefs: [(13, 'Feedr'), (25, '760'), (25, '375'), (13, 'PosA'), (62, 'Ex'), (20, 'Shed'), (73, 'Gar2'), (21, 'Membran'), (71, '0'), (25, '816')]
mse: 4.39319368472891
intercept: 7.98000843582376
predictions after exponentiation: [(115463.21896144339, 117999.99999999991), (27993195858883.7, 325300.0000000001), (136694.0663055813, 149349.99999999988), (177823.61777403223, 152999.99999999994), (156811.5297447705, 170000.0), (96623.18792454217, 109999.99999999991), (1676000.0143689625, 180499.99999999988), (115870.19646859728, 136500.0000000001), (188627.3230280837, 173999.99999999997), (114012.5253